In [1]:
import os
import sys
import torch
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    #quantization_config=bnb_config, # Закомментировать в случае использования небольших моделей, где не нужна квантизация
    device_map={"": "cuda"}, # В случае нескольких GPU нужно указать конкретную, cuda:0, например
    attn_implementation="sdpa", #attn_implementation='flash_attention_2' для GPU, которые поддерживают
)
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [2]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

In [3]:
generation_config.temperature = 0.3
generation_config.max_new_tokens = 512
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 512,
  "temperature": 0.3
}

In [4]:
print(tokenizer.apply_chat_template([{'role': 'user', 'content': 'Что такое ИИ?'}], tokenize=False, add_generation_prompt=True))

<s> [INST] Что такое ИИ? [/INST]


In [5]:
print(tokenizer.chat_template)

{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content'] %}
    {%- set loop_messages = messages[1:] %}
{%- else %}
    {%- set loop_messages = messages %}
{%- endif %}

{{- bos_token }}
{%- for message in loop_messages %}
    {%- if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}
        {{- raise_exception('After the optional system message, conversation roles must alternate user/assistant/user/assistant/...') }}
    {%- endif %}
    {%- if message['role'] == 'user' %}
        {%- if loop.first and system_message is defined %}
            {{- ' [INST] ' + system_message + '\n\n' + message['content'] + ' [/INST]' }}
        {%- else %}
            {{- ' [INST] ' + message['content'] + ' [/INST]' }}
        {%- endif %}
    {%- elif message['role'] == 'assistant' %}
        {{- ' ' + message['content'] + eos_token}}
    {%- else %}
        {{- raise_exception('Only user and assistant roles are supported, with the exception of an initial opt

In [6]:
prompts = [
    {'role': 'user', 'content': 'Объясни квантовую механику простыми словами'},
    {'role': 'user', 'content': 'Напиши короткий рассказ о космосе'},
    {'role': 'user', 'content': 'Составь рецепт борща'},
    {'role': 'user', 'content': 'Объясни как работает блокчейн'},
    {'role': 'user', 'content': 'Сделай краткое резюме романа "Преступление и наказание"'},
    {'role': 'user', 'content': 'Придумай сборку на акса в доте 2'},
    {'role': 'user', 'content': 'Напиши поздравление с днем рождения для Артура'},
    {'role': 'user', 'content': 'Объясни как работает градиентный спуск'},
    {'role': 'user', 'content': 'Сделай список идей для стартапа в сфере образования'},
    {'role': 'user', 'content': 'Напиши шутку про программистов'}
]

In [7]:
import torch

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

input_ids = tokenizer.apply_chat_template(
    [[prompt] for prompt in prompts],
    add_generation_prompt=True,
    tokenize=True,
    padding=True,
    return_tensors='pt'
)

input_ids = input_ids.to(model.device)
attention_mask = (input_ids != tokenizer.pad_token_id).int()

outputs = model.generate(
    input_ids,
    attention_mask=attention_mask,
    generation_config=generation_config
)

generated_responses = []
for i, output in enumerate(outputs):
    original_length = attention_mask[i].sum().item()
    generated_tokens = output[original_length:].detach().cpu()
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    generated_responses.append(response)

for i, (prompt, response) in enumerate(zip(prompts, generated_responses)):
    print(f"Промпт {i+1}: {prompt['content']}")
    print(f"Ответ {i+1}: {response}")
    print("-" * 50)

C:\Users\artur\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Промпт 1: Объясни квантовую механику простыми словами
Ответ 1: ми [/INST] Квантовая механика - это теория, которая описывает поведение материальных частиц на уровне атомов и subatomic (атомных) частиц, таких как электроны и фотоны.

В классической физике мы можем предсказать положение и скорость объекта в любой момент времени, используя законы механики и электромагнетизма. Но в квантовой механике делается несколько иное предположение.

1. Энергия и импульс (соответствующий моменту импульса) могут принимать только определенные дискретные значения. Это называется квантованием энергии.
2. Суперпозиция: Квантовая система может находиться в нескольких состояниях одновременно, пока мы не наблюдаем её. Это называется суперпозицией состояний.
3. Коллапс волновой функции: Когда мы наблюдаем систему, то она переходит в одно из возможных состояний. Это называется коллапсом волновой функции.

Квантовая механика может показать необычные эффекты, такие как эффект туннелирования, эффект спин-орбиталь